In [2]:
import pandas as pd, numpy as np
np.set_printoptions(suppress=True)# 关掉科学计数法
import glob
import os
import csv
# 一次性merge多个pct_chg
from functools import reduce
from datetime import datetime, timedelta

# import yfinance as yf
import tushare as ts
import time, urllib
ts.set_token('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go

#setting offilne
plotly.offline.init_notebook_mode(connected=True)

# Paramater Settings

In [4]:
etfs_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"

In [108]:
# Get ETF tickers from the summary data sheet
def get_all_etf_tickers_from_summary_file(etf_summary_file):
    ch_etfs_df = pd.read_csv(etf_summary_file, engine="python")
    ch_etfs_df['code'] =ch_etfs_df['证券代码'].apply(lambda x: str(x)[:6])
    ch_etfs_ticker = list(ch_etfs_df['code'].unique())
    return ch_etfs_ticker

#=============================================================
# Get all market data columns as a dataframe from a folder
def get_mkt_data_df(path, ticker_list, date_col):
    # e.g. ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
    csv_path = path+"*.csv"
    files = glob.glob(csv_path)
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df_list.append(ticker_df) 
        except Exception as e:
            print(e)
    try:
        tickers_data_concated = pd.concat(ticker_df_list)
        tickers_data_concated.reset_index(inplace=True)
        del tickers_data_concated['index']  
    except Exception as e:
        print(e)
    return tickers_data_concated

# Get date_col, price_col, code
def get_date_price_code_df(path, ticker_list, date_col, price_col, code_col):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df = ticker_df[[date_col, price_col, code_col]]
            ticker_df['code'] = ticker
            ticker_df_list.append(ticker_df)
        except Exception as e:
            print(e)
    try:
        tickers_data_concated = pd.concat(ticker_df_list)
        tickers_data_concated.reset_index(inplace=True)
        del tickers_data_concated['index']  
    except Exception as e:
        print(e)
    return tickers_data_concated

# Get date_col, price_col, code_col, pct_chg_col
def get_date_price_code_return_df(path, ticker_list, date_col, price_col, code_col):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df = ticker_df[[date_col, price_col, code_col]]
            ticker_df['pct_chg'] = ticker_df[price_col].pct_change()
            ticker_df = ticker_df[[date_col, 'pct_chg']].dropna()
            ticker_df['code'] = ticker
            ticker_df_list.append(ticker_df)
        except Exception as e:
            print(e)
    try:
        tickers_data_concated = pd.concat(ticker_df_list)
        tickers_data_concated.reset_index(inplace=True)
        del tickers_data_concated['index']  
    except Exception as e:
        print(e)
    return tickers_data_concated

# Get date_col, price_col, code_col, pct_chg_col
def get_date_price_code_cumreturns_df(path, ticker_list, date_col, price_col, code_col):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df = ticker_df[[date_col, price_col, code_col]]
            ticker_df['pct_chg'] = ticker_df[price_col].pct_change()
            ticker_df['cum_returns'] = ticker_df['pct_chg'].cumsum()+1
            ticker_df = ticker_df[[date_col, 'cum_returns']].dropna()
            ticker_df['code'] = ticker
            ticker_df_list.append(ticker_df)
        except:
            pass
    try:
        tickers_data_concated = pd.concat(ticker_df_list)
    except Exception as e:
        print(e)
    return tickers_data_concated

#=============================================================

# Get all market data columns as a dataframe from a folder
def get_mkt_data_list(path, ticker_list, date_col):
    # e.g. ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
    csv_path = path+"*.csv"
    files = glob.glob(csv_path)
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df_list.append(ticker_df) 
        except Exception as e:
            print(e)
    return ticker_df_list

# Get date_col, price_col, code
def get_date_price_code_list(path, ticker_list, date_col, price_col, code_col):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df = ticker_df[[date_col, price_col]]
            ticker_df.columns = [date_col, ticker]
            ticker_df_list.append(ticker_df)
        except Exception as e:
            print(e)
    return ticker_df_list

# Get date_col, price_col, code_col, pct_chg_col
def get_date_price_code_return_list(path, ticker_list, date_col, price_col, code_col):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df = ticker_df[[date_col, price_col, code_col]]
            ticker_df['pct_chg'] = ticker_df[price_col].pct_change()
            ticker_df = ticker_df[[date_col, 'pct_chg']].dropna()
            ticker_df.columns = [date_col, ticker]
            ticker_df_list.append(ticker_df)
        except Exception as e:
            print(e)
    return ticker_df_list

# Get date_col, price_col, code_col, pct_chg_col
def get_date_price_code_cumreturns_list(path, ticker_list, date_col, price_col, code_col):
    # for etf data cols are 'date', 'close', 'code'
    ticker_df_list = []
    for ticker in ticker_list:
        try:
            ticker_df = pd.read_csv(path+ticker+".csv")
            ticker_df = ticker_df.sort_values(date_col)
            ticker_df = ticker_df[[date_col, price_col, code_col]]
            ticker_df['pct_chg'] = ticker_df[price_col].pct_change()
            ticker_df['cum_returns'] = ticker_df['pct_chg'].cumsum()+1
            ticker_df = ticker_df[[date_col, 'cum_returns']].dropna()
            ticker_df.columns = [date_col, ticker]
            ticker_df_list.append(ticker_df)
        except:
            pass
    return ticker_df_list

#=============================================================
# merge df1 with df2
def merge_df_for_reduce(df1, df2, date_col="date"):
    # By default the etf's date_col goes by 'date'
    merged = df1.merge(df2, on = date_col, how = 'outer')
    merged.sort_values(date_col, inplace = True)
    return merged

# merge a list of dfs on date_col
def merge_dfs_by_ticker(ticker_df_list, date_col):
    merged_all = reduce(merge_df_for_reduce, ticker_df_list)
    merged_all.set_index(date_col, inplace=True)
    merged_all.dropna(how="all", axis = 1, inplace = True)
    merged_all.fillna(method="ffill", inplace = True)
    return merged_all


def get_sector_leaders():
    etf_sectors = pd.read_csv("/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/ETF_板块已分类.csv")
    etf_sectors.dropna(inplace=True)
    filtered_1 = etf_sectors[etf_sectors['所属板块']!=""]
    filtered_1.columns = ['ticker','name','type1','type2','mkt_cap','institution_holdings',
                          'ins_holding_pct','fee1%','fee2%','fee3%','sector']
    filtered_1['mkt_cap'] = filtered_1['mkt_cap'].apply(lambda x: x.replace(",","")).apply(lambda x: float(x))
    filtered_1.reset_index(inplace = True)
    del filtered_1['index']
    filtered_1['rank'] = filtered_1.groupby(['sector'])['mkt_cap'].rank()
    filtered_2 = filtered_1[filtered_1['rank']==1]
    filtered_2['code'] = filtered_2['ticker'].str.split(".", expand = True)[0]
    filtered_2['exchange'] = filtered_2['ticker'].str.split(".", expand = True)[1]
    filtered_2_tickers = list(filtered_2['code'].unique())
    
    return filtered_2_tickers

def MACD(df, n_fast, n_slow, n_macd, price_col): # n_fast = 12, n_slow = 26
    """
    http://stockcharts.com/docs/doku.php?id=scans:indicators
    MACD, MACD Signal and MACD difference, rationale CHECKED, code CHECKED, updated
    # Conventional look-back window for calculating MACDsign is 9
    """
    EMAfast = df[price_col].ewm(span = n_fast, min_periods = n_fast - 1).mean()
    EMAslow = df[price_col].ewm(span = n_slow, min_periods = n_slow - 1).mean()
    MACD = pd.Series(EMAfast - EMAslow, name = 'MACD_' + str(n_fast) + '_' + str(n_slow))
    MACDsign = MACD.ewm(span = n_macd, min_periods = n_macd-1).mean().rename('MACDsign_' + str(n_fast) + '_' + str(n_slow))
    MACDdiff = pd.Series(MACD - MACDsign, name = 'MACDdiff_' + str(n_fast) + '_' + str(n_slow))
    df['MACD_Diff'] = MACD
    df['MACD_Diff_EMA'] = MACDsign
    df['MACD'] = MACDdiff
    df['SIGNAL_STATUS'] = df['MACD'].apply(lambda x: "多头状态" if x>0 else ("空头状态" if x<0 else "无信号状态"))
    return df

def calc_macd_signals(tickers_data_concated, ticker_list, code_col, ticker_type, price_col):
    signal_record = []
    signal_data = []
    if len(ticker_list)!=1:
        for ticker in ticker_list:
            print("Calculating Signal for: ", ticker)
            try:
                if ticker_type == "float":
                # Be aware of types of ticker values here, whether it's float or strings, depends.
                    single_ticker_df = tickers_data_concated[tickers_data_concated[code_col]==float(ticker)]
                elif ticker_type == "string":
                    single_ticker_df = tickers_data_concated[tickers_data_concated[code_col]==ticker]
                signal_df = MACD(single_ticker_df, 12, 26, 9, price_col)
#                 last_signal = signal_df[[code_col,'SIGNAL_STATUS']].values[-1]
                signal_data.append(signal_df)
#                 signal_record.append(last_signal)
            except:
                pass
        signal_data_df = pd.concat(signal_data)
    else:
        print("Calculating Signal for: ", ticker_list[0])
        try:                
            print("getting thru")
            signal_df = MACD(tickers_data_concated, 12, 26, 9, price_col)
#             last_signal = signal_df[[code_col,'SIGNAL_STATUS']].values[-1]
#             signal_data.append(signal_df)
#             signal_record.append(last_signal)
        except:
            pass
        signal_data_df = signal_df
    return signal_data_df

def make_numeric_signals(series):
    for item in series:
#         print(item)
        if item =="多":
            return 1
        elif item =="空":
            return -1
        else:
            return 0
        
def generate_most_recent_signal_data_v1(signal_data_df, date_col, code_col):
    # v1 is the version of generating the og macd signals
    signal_data_df['SIGNAL_DIRECTION'] = signal_data_df['SIGNAL_STATUS'].apply(lambda x: make_numeric_signals(x))
    signal_data_df['SIGNAL_DIRECTION_DIFF'] = signal_data_df.groupby([code_col])['SIGNAL_DIRECTION'].diff()
    signal_data_df['SIGNAL_ACTION'] = signal_data_df['SIGNAL_DIRECTION_DIFF'].apply(lambda x: "LONG" if x==2 else("SHORT" if x==-2 else "NO CHANGE"))
    most_recent_signals = signal_data_df.groupby([code_col])[[date_col,code_col,'SIGNAL_STATUS','SIGNAL_ACTION']].tail(1)
    return most_recent_signals

def merge_current_pos_with_target_pos(etf_path_to_csv, cur_positions, tgt_most_recent_etf_macd_signals):
    cur_pos_prices = select_df_from_db_by_tickers(etf_path_to_csv, cur_positions)
    cur_pos_etf_macd_signals = calc_macd_signals(cur_pos_prices, cur_positions, 'code', 'float','close')
    most_recent_cur_pos_etf_macd_signals = generate_most_recent_signal_data_v1(cur_pos_etf_macd_signals, 'date','code')
    most_recent_cur_pos_etf_macd_signals['TYPE'] = 'CUR_POS'
    all_macd_signal_df = most_recent_cur_pos_etf_macd_signals.merge(tgt_most_recent_etf_macd_signals, on = ['date','code'], how = 'outer')
    return all_macd_signal_df

In [31]:
get_date_price_code(etfs_path, ['159941','510010'], 'date','close','code')

,date,close,code
0,2015-07-13,0.958,159941
1,2015-07-14,0.967,159941
2,2015-07-15,0.963,159941
3,2015-07-16,0.966,159941
4,2015-07-17,0.989,159941
...,...,...,...
2457,2020-02-24,1.170,510010
2458,2020-02-25,1.165,510010
2459,2020-02-26,1.158,510010
2460,2020-02-27,1.160,510010


In [42]:
get_mkt_data(etfs_path, ['159941','159926'],'date')

,date,open,close,high,low,volume,code
0,2015-07-13,0.950,0.958,0.987,0.902,529343.0,159941
1,2015-07-14,0.970,0.967,0.975,0.953,286423.0,159941
2,2015-07-15,0.963,0.963,0.980,0.960,164837.0,159941
3,2015-07-16,0.963,0.966,0.970,0.961,109387.0,159941
4,2015-07-17,0.968,0.989,0.999,0.968,64955.0,159941
...,...,...,...,...,...,...,...
2305,2020-02-24,113.996,113.500,114.000,112.006,18.0,159926
2306,2020-02-25,112.340,112.989,112.989,112.340,2.0,159926
2307,2020-02-26,113.899,112.280,113.900,112.280,4.0,159926
2308,2020-02-27,114.389,113.980,114.389,113.980,4.0,159926


In [43]:
get_date_price_code_returns(etfs_path, ['159941','510100','518880','159926'], 'date', 'close', 'code')

/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:69: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




,159926,159941,510100,518880,date
1,NaN,0.009395,NaN,NaN,2015-07-14
2,NaN,-0.004137,NaN,NaN,2015-07-15
3,NaN,0.003115,NaN,NaN,2015-07-16
4,NaN,0.023810,NaN,NaN,2015-07-17
5,NaN,0.001011,NaN,NaN,2015-07-20
...,...,...,...,...,...
1177,0.005395,NaN,NaN,NaN,2020-02-24
1178,-0.004502,NaN,NaN,NaN,2020-02-25
1179,-0.006275,NaN,NaN,NaN,2020-02-26
1180,0.015141,NaN,NaN,NaN,2020-02-27


In [66]:
merge_dfs_by_ticker(ticker_df_list, date_col)

NameError: name 'merge_dfs_by_ticker' is not defined

In [103]:
get_date_price_code_cumreturns_list(etfs_path, ['159941','510100','518880','159926'], 'date', 'close', 'code')

[            date    159941
 1     2015-07-14  1.009395
 2     2015-07-15  1.005258
 3     2015-07-16  1.008373
 4     2015-07-17  1.032183
 5     2015-07-20  1.033194
 ...          ...       ...
 1123  2020-02-24  1.868154
 1124  2020-02-25  1.885773
 1125  2020-02-26  1.856760
 1126  2020-02-27  1.856278
 1127  2020-02-28  1.812884
 
 [1127 rows x 2 columns],           date    510100
 1   2019-10-10  1.004985
 2   2019-10-11  1.019866
 3   2019-10-14  1.031596
 4   2019-10-15  1.031596
 5   2019-10-16  1.027731
 6   2019-10-17  1.029671
 7   2019-10-18  1.015151
 8   2019-10-21  1.018097
 9   2019-10-22  1.017118
 10  2019-10-23  1.010255
 11  2019-10-24  1.012230
 12  2019-10-25  1.016171
 13  2019-10-28  1.020096
 14  2019-10-29  1.016186
 15  2019-10-30  1.013242
 16  2019-10-31  1.012258
 17  2019-11-01  1.029006
 18  2019-11-04  1.035789
 19  2019-11-05  1.040602
 20  2019-11-06  1.038686
 21  2019-11-07  1.040605
 22  2019-11-08  1.033900
 23  2019-11-11  1.018471
 24  2019-11-

In [94]:
get_date_price_code_cumreturns_df(etfs_path, ['159941','510100','518880','159926'], 'date','close','code')

,date,cum_returns,code
1,2015-07-14,1.009395,159941
2,2015-07-15,1.005258,159941
3,2015-07-16,1.008373,159941
4,2015-07-17,1.032183,159941
5,2015-07-20,1.033194,159941
...,...,...,...
1177,2020-02-24,1.255399,159926
1178,2020-02-25,1.250897,159926
1179,2020-02-26,1.244622,159926
1180,2020-02-27,1.259763,159926


In [122]:
dt_price_code_list = get_date_price_code_list(etfs_path, ['510100'], 'date','close','code')

In [123]:
dt_price_code_list

[          date  510100
 0   2019-10-09   1.003
 1   2019-10-10   1.008
 2   2019-10-11   1.023
 3   2019-10-14   1.035
 4   2019-10-15   1.035
 5   2019-10-16   1.031
 6   2019-10-17   1.033
 7   2019-10-18   1.018
 8   2019-10-21   1.021
 9   2019-10-22   1.020
 10  2019-10-23   1.013
 11  2019-10-24   1.015
 12  2019-10-25   1.019
 13  2019-10-28   1.023
 14  2019-10-29   1.019
 15  2019-10-30   1.016
 16  2019-10-31   1.015
 17  2019-11-01   1.032
 18  2019-11-04   1.039
 19  2019-11-05   1.044
 20  2019-11-06   1.042
 21  2019-11-07   1.044
 22  2019-11-08   1.037
 23  2019-11-11   1.021
 24  2019-11-12   1.024
 25  2019-11-13   1.023
 26  2019-11-14   1.023
 27  2019-11-15   1.019
 28  2019-11-18   1.032
 29  2019-11-19   1.038
 30  2019-11-20   1.027
 31  2019-11-21   1.021
 32  2019-11-22   1.012
 33  2019-11-25   1.021
 34  2019-11-26   1.021
 35  2019-11-27   1.020
 36  2019-11-28   1.015
 37  2019-11-29   1.002
 38  2019-12-02   1.004
 39  2019-12-03   1.008
 40  2019-12-04 

In [119]:
merge_dfs_by_ticker(dt_price_code_list, 'date')

,159941,510100,518880,159926
date,,,,
2013-07-30,NaN,NaN,1.007616,NaN
2013-07-31,NaN,NaN,1.011773,NaN
2013-08-01,NaN,NaN,1.006504,NaN
2013-08-02,NaN,NaN,0.981911,NaN
2013-08-05,NaN,NaN,1.002082,NaN
...,...,...,...,...
2020-02-24,1.868154,1.041377,1.399013,1.255399
2020-02-25,1.885773,1.041377,1.378342,1.250897
2020-02-26,1.856760,1.041377,1.383825,1.244622
